# C - 柱柱柱柱柱
実行時間制限: 2 sec / メモリ制限: 256 MB

問題文
N
 本の木の柱が左から右へ一列に並んだアスレチックがあります。左から 
i
 本目の柱の高さは 
a
i
 センチメートルです。

高橋君は左から 
1
 本目の柱からスタートし、右へ柱を渡っていき 
N
 本目の柱まで行こうとしています。

高橋君がある柱にいるとき、次には現在の柱から 
1
 個もしくは 
2
 個右にある柱のどちらかへ移動することができます。

移動するときには、現在いる柱の高さと、移動後の柱の高さの差の絶対値のぶんだけコストがかかります。

N
 本目の柱まで行くとき、コストの合計の最小値はいくらになるでしょうか。

## ~~再帰関数(深さ優先探索)~~分割統治法を使う

In [30]:
import sys

n = int(input())
heights = list(map(lambda x: int(x), input().split()))

def cost_func(current_height, remaining_heights):
    if len(remaining_heights) == 1:
        return abs(remaining_heights[0] - current_height)
    if len(remaining_heights) == 2:
        cost_with_1step = abs(remaining_heights[1] - current_height)
        cost_with_2steps = abs(remaining_heights[0] - current_height) + abs(remaining_heights[1] - current_height)
        return min(cost_with_1step, cost_with_2steps)
        
    cost_after_one_step = abs(remaining_heights[0] - current_height) + cost_func(remaining_heights[0], remaining_heights[1:])
    cost_after_two_step = abs(remaining_heights[1] - current_height) + cost_func(remaining_heights[1], remaining_heights[2:])
    return min(cost_after_one_step, cost_after_two_step)

print(cost_func(heights[0], heights[1:]))

9
314 159 265 358 979 323 846 264 338
310


## 探索を効率化する　(メモ化 + 工夫)

In [59]:
import sys
from functools import lru_cache
sys.setrecursionlimit(20000) # 再起呼び出しの最大数をセット

import time
t = time.time()
n = int(input())
heights = tuple(map(lambda x: int(x), input().split()))

# 関数をメモ化
@lru_cache(maxsize=None)
def cost_func(current_height, remaining_heights):
    if len(remaining_heights) == 1:
        return abs(remaining_heights[0] - current_height)
    if len(remaining_heights) == 2:
        cost_with_1step = abs(remaining_heights[1] - current_height)
        cost_with_2steps = abs(remaining_heights[0] - current_height) + abs(remaining_heights[1] - current_height)
        return min(cost_with_1step, cost_with_2steps)
    
    # 短調増加中は全部一歩で進む
#     # (1) 2 3 4 5 -10　だと　i=4でエラー、i=2までは短調増加でどちらでok
#     first_gap = remaining_heights[0] - current_height
#     for i in range(1, len(remaining_heights)):
#         if  (remaining_heights[i] - remaining_heights[i-1]) * first_gap <= 0:
#             break
#     if i >= 3:
#         return remaining_heights[i-1] - current_height + cost_func(remaining_heights[i-1], remaining_heights[i-2:])
    if len(remaining_heights) >= 3:
        if (current_height < remaining_heights[0]) & (remaining_heights[0] < remaining_heights[1]) & (remaining_heights[1] < remaining_heights[2]):
            for i in range(len(remaining_heights)-3):
                if remaining_heights[i+2] < remaining_heights[i+3]:
                    break
            if i > 0:
                return abs(remaining_heights[i] - current_height) + cost_func(remaining_heights[i], remaining_heights[i+1:])
        elif (current_height > remaining_heights[0]) & (remaining_heights[0] > remaining_heights[1]) & (remaining_heights[1] > remaining_heights[2]):
            for i in range(len(remaining_heights)-3):
                if remaining_heights[i+2] > remaining_heights[i+3]:
                    break
            if i > 0:
                return abs(remaining_heights[i] - current_height) + cost_func(remaining_heights[i], remaining_heights[i+1:])
    
        
    cost_after_one_step = abs(remaining_heights[0] - current_height) + cost_func(remaining_heights[0], remaining_heights[1:])
    cost_after_two_step = abs(remaining_heights[1] - current_height) + cost_func(remaining_heights[1], remaining_heights[2:])
    return min(cost_after_one_step, cost_after_two_step)

print(cost_func(heights[0], heights[1:]))
# print(time.time() - t)

3
314 159 265 358 979 323 846 264 338
310
9.147440910339355


## 動的計画法(DP)でいけた


[リンク先の画像](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F182963%2F32c1c785-f7ad-cb18-3b1d-e34d20cec6b7.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=a736fce3cf33e34ca41e48a1c27894f7)を参考に。

In [68]:
n = int(input())
heights = list(map(lambda x: int(x), input().split()))

costs = [0, abs(heights[1] - heights[0])]

for i in range(2, n):
    costs.append(min(costs[i-2] + abs(heights[i] - heights[i-2]), costs[i-1] + abs(heights[i] - heights[i-1])))

print(costs[-1])

9
314 159 265 358 979 323 846 264 338
310


# 考察

- 結局DPで一瞬だった。果たして深さ優先探索って本当に再帰+メモ化でやらないといけないパターンはあるのか？

## 深さ優先探索(再帰)と動的計画法の違いは？

- なんとなく関係している気がするが、これらの概念を今はうまく結び付けて説明することはできない(またいつか)


- 深さ優先探索していく上で、全ての経路を探索する必要はない、つまり、ある深さのnodeに達した時の、その深さまでのなんらかの計算値が最小になる場合のみを考えるとする。と、深く深く進んでいくと同時に、その深さで最小のpathだけを考えて、その続きのみDFSしていけばいいことになる。これがDPが深さ優先探索の絞られたバージョンである感覚を自分が受ける理由ではないだろうか。


## 貪欲法と動的計画法の違いは？

- 貪欲法はその瞬間に得する方法を選択する。つまり今回の問題なら自分の今の位置から見て１個先か２個先のうちコストが低い方を選んで飛んで一歩一歩進んでいく。
- 動的計画法は過去の選択の結果を考慮して一歩一歩進んでいく。
- 深さ優先探索は総当たりのため、後戻りもする。